# Initializations


In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os
from numpy.random import shuffle
import shutil
import patchify as p
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time
from matplotlib import cm
import segmentation_models as sm

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

# Hyperparameter Definition

In [ ]:
img_w, img_h = 512, 512

# Optimization params
bs = 4
lr = 1e-3
decay = 0.1
min_lr = 1e-6

apply_data_augmentation = True

mode = "bipbip"
plant = "haricot"

BACKBONE = "custom"

loss_name = "loss"

# Script Comprehension

Converting in classes


In [ ]:
# Converts images to RGB to Target
def read_rgb_mask(img_path):
    '''
    img_path: path to the mask file
    Returns the numpy array containing target values
    '''

    mask_img = Image.open(img_path)
    mask_arr = np.array(mask_img)

    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)

    # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
    new_mask_arr[np.where(np.all(mask_arr == [254, 124, 18], axis=-1))] = 0
    new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
    new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2

    return new_mask_arr

def mask_to_rgb(mask_arr):

    mask = np.squeeze(mask_arr, axis=2)
    new_mask_arr = np.zeros((mask_arr.shape[0], mask_arr.shape[1], 3), dtype=mask_arr.dtype)

    new_mask_arr[np.where(mask == 0)] = np.array([0, 0, 0])
    new_mask_arr[np.where(mask == 1)] = np.array([255, 255, 255])
    new_mask_arr[np.where(mask == 2)] = np.array([216, 67, 82])

    return new_mask_arr

RLE Encoding

In [ ]:
def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

Script for dividing training set and validation set. Create .txts.

In [ ]:
# plant = 'haricot' OR 'mais' OR 'all'
# mode = 'all' OR 'bipbip' OR 'pead' OR 'roseau' OR 'weedelec'
def split_set(base_dir = './Development_Dataset/Training', split = 0.2, plant = 'haricot', mode = 'all', is_test = False):
    
    plants = []
    
    if plant == 'all':
        plants = ['haricot', 'mais']
    else:
        plants.append(plant)
    
    if not is_test:
      datasets = []
      files_ls = []
    
      if mode == 'all':
          datasets.extend(['Bipbip', 'Pead', 'Roseau', 'Weedelec'])
      else:
          datasets.append(mode.capitalize())
        
      for dataset in datasets:
        for p in plants:
            images_dir = os.path.join(base_dir, dataset, p.capitalize(), 'Images')
            files_ls_temp = os.listdir(images_dir)
            files_ls.extend([filename.split('.')[0] + '\n' for filename in files_ls_temp])
            
      shuffle(files_ls)
          
      final_element = int(len(files_ls) * split)
      val_ls = files_ls[:final_element]
      train_ls = files_ls[final_element:]
      print('# images for training: ' + str(len(train_ls)))
      print('# images for validation: ' + str(len(val_ls)))
          
      with open(os.path.join(base_dir,"train.txt"), "w") as train_txt:
          train_txt.writelines(train_ls)
          
      with open(os.path.join(base_dir,"val.txt"), "w") as val_txt:
          val_txt.writelines(val_ls)
    
    else:

      datasets = []
      datasets.extend(['Bipbip', 'Pead', 'Roseau', 'Weedelec'])

      path = os.path.join(base_dir, "temptest")
      if os.path.exists(path):
        shutil.rmtree(path)
      os.mkdir(path)
    
      plants = ['haricot', 'mais']

      for pl in plants:
        for dataset in datasets:
          images_dir = os.path.join(base_dir, dataset, pl.capitalize(), 'Images')
          files_ls_temp = os.listdir(images_dir)
          for file in files_ls_temp:
            shutil.copy(os.path.join(images_dir, file), path)

Code for plotting data

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["iou_score"])
    plt.plot(hist.history["val_iou_score"])
    plt.plot(hist.history[loss_name])
    plt.plot(hist.history["val_" + loss_name])
    plt.title("model IoU")
    plt.ylabel("score")
    plt.xlabel("epoch")
    plt.legend(["train_IoU", "val_IoU", loss_name, "val_" + loss_name], loc="upper left")
    plt.show()

# Data Preparation

In [ ]:
cwd = './'

dataset_dir = "Development_Dataset"
training_dir = os.path.join(dataset_dir, "Training")
test_dir = os.path.join(dataset_dir, "Test_Dev")

In [ ]:
# Split train and validation sets
split_set(base_dir= training_dir, mode=mode, plant=plant)

Dictionary with array colors, used for visualization

In [ ]:
dict_rgb = {
    "background" : [254, 124, 18],
    "crop" : [255, 255, 255],
    "weed" : [216, 67, 82]
}

Generator for creating imgs from patch dataset

In [ ]:
class CustomDataset(tf.keras.utils.Sequence):

  def __init__(self, dataset_dir, which_subset, img_generator=None, mask_generator=None, 
               preprocessing_function=None, out_shape=[img_w, img_h]):
    if which_subset == 'training':
      subset_file = os.path.join(dataset_dir, 'train.txt')
    elif which_subset == 'validation':
      subset_file = os.path.join(dataset_dir, 'val.txt')
    
    with open(subset_file, 'r') as f:
      lines = f.readlines()
    
    subset_filenames = []
    for line in lines:
      subset_filenames.append(line.strip()) 

    self.which_subset = which_subset
    self.dataset_dir = dataset_dir
    self.subset_filenames = subset_filenames
    self.img_generator = img_generator
    self.mask_generator = mask_generator
    self.preprocessing_function = preprocessing_function
    self.out_shape = out_shape

  def __len__(self):
    return len(self.subset_filenames)

  def __getitem__(self, index):
    
    curr_filename = self.subset_filenames[index]
    
    split_filename = curr_filename.split('_')
    curr_dataset = split_filename[0]
    curr_plant = split_filename[1]
    
    if curr_dataset == 'Roseau':
        img = Image.open(os.path.join(self.dataset_dir, curr_dataset, curr_plant.capitalize(), 'Images',
                                      curr_filename + '.png'))
    else:
        img = Image.open(os.path.join(self.dataset_dir, curr_dataset, curr_plant.capitalize(), 'Images',
                                      curr_filename + '.jpg'))
    
    mask = Image.fromarray(read_rgb_mask(os.path.join(self.dataset_dir, curr_dataset, curr_plant.capitalize(),
                                                      'Masks', curr_filename + '.png')))

    img = img.resize(self.out_shape)
    mask = mask.resize(self.out_shape, resample=Image.NEAREST)

    img_arr = np.array(img)
    mask_arr = np.array(mask)

    mask_arr = np.expand_dims(mask_arr, -1)
    
    out_mask = mask_arr
    
    if self.which_subset == 'training':
      if self.img_generator is not None and self.mask_generator is not None:
        
        img_t = self.img_generator.get_random_transform(img_arr.shape)
        img_arr = self.img_generator.apply_transform(img_arr, img_t)
        
        out_mask = np.zeros_like(mask_arr)
        for c in np.unique(mask_arr):
          if c > 0:
            curr_class_arr = np.float32(mask_arr == c)
            curr_class_arr = self.mask_generator.apply_transform(curr_class_arr, img_t)
            
            curr_class_arr = np.uint8(curr_class_arr)
            
            curr_class_arr = curr_class_arr * c 
            out_mask += curr_class_arr
    else:
      out_mask = mask_arr
    
    if self.preprocessing_function is not None:
        img_arr = self.preprocessing_function(img_arr)

    return img_arr, np.float32(out_mask)

Generators and Datasets

In [ ]:
# Create training ImageDataGenerator object
# The commented transformations seemed to hurt performance
if apply_data_augmentation:
    img_data_gen = ImageDataGenerator(# rotation_range=10,
                                      # width_shift_range=10,
                                      # height_shift_range=10,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect')
    mask_data_gen = ImageDataGenerator(# rotation_range=10,
                                       # width_shift_range=10,
                                       # height_shift_range=10,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       fill_mode='reflect')
else:
    img_data_gen = ImageDataGenerator()
    mask_data_gen = ImageDataGenerator()

In [ ]:
preprocess_input = None

dataset = CustomDataset(training_dir, 'training', 
                        img_generator=img_data_gen, mask_generator=mask_data_gen,
                        preprocessing_function=preprocess_input)
dataset_valid = CustomDataset(training_dir, 'validation', preprocessing_function=preprocess_input)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: dataset,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 1]))

train_dataset = train_dataset.batch(bs)

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: dataset_valid,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 1]))
valid_dataset = valid_dataset.batch(bs)

valid_dataset = valid_dataset.repeat()

Visualize image and mask to be sure that all works well

In [ ]:
iterator = iter(train_dataset)

fig, ax = plt.subplots(1, 2)

augmented_img, target = next(iterator)
augmented_img = augmented_img[0]
augmented_img = augmented_img

print(augmented_img.shape, target.shape)
target = np.array(target[0, ..., 0])

print(np.unique(target))

target_img = np.zeros([target.shape[0], target.shape[1], 3])

target_img[np.where(target == 0)] = [0, 0, 0]

class_names = ["background", "crop", "weed"]
for i in range(1, 3):
  target_img[np.where(target == i)] = np.array(dict_rgb[class_names[i]])

ax[0].imshow(np.uint8(augmented_img))
ax[1].imshow(np.uint8(target_img))

plt.show()

## Convolutional Neural Network (CNN)
### Encoder-Decoder

Preparing model

In [ ]:
def conv_block(input_tensor, num_filters):
  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
  encoder = layers.BatchNormalization()(encoder)
  encoder = layers.Activation('relu')(encoder)
  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
  encoder = layers.BatchNormalization()(encoder)
  encoder = layers.Activation('relu')(encoder)
  return encoder

def encoder_block(input_tensor, num_filters):
  encoder = conv_block(input_tensor, num_filters)
  encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
  
  return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
  decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
  decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  return decoder

In [ ]:
from tensorflow.python.keras import layers
from tensorflow.python.keras import models

def create_model(depth, start_f, num_classes, img_shape = (224,224,3)):
    inputs = layers.Input(shape=img_shape)
    # 256

    encoder0_pool, encoder0 = encoder_block(inputs, 32)
    # 128

    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
    # 64

    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
    # 32

    encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)
    # 16

    encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512)
    # 8

    center = conv_block(encoder4_pool, 1024)
    # center

    decoder4 = decoder_block(center, encoder4, 512)
    # 16

    decoder3 = decoder_block(decoder4, encoder3, 256)
    # 32

    decoder2 = decoder_block(decoder3, encoder2, 128)
    # 64

    decoder1 = decoder_block(decoder2, encoder1, 64)
    # 128

    decoder0 = decoder_block(decoder1, encoder0, 32)
    # 256

    outputs = layers.Conv2D(3, (1, 1), activation='softmax')(decoder0)
    model = models.Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [ ]:
model = create_model(depth=5, 
                     start_f=8, 
                     num_classes=3,
                     img_shape = (img_w,img_h,3))

model.summary()

Dice Coefficient as Loss Function

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + 1e-8) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1e-8)

def dice_loss(y_true, y_pred):
    dice=0
    y_true = tf.squeeze(y_true, -1)
    for index in range(1, 3):
        dice -= dice_coef(tf.cast(tf.where(y_true == index, 1, 0), tf.float32), tf.cast(y_pred[:,:,:,index], tf.float32))
    return dice

Losses, Metrics and Optimizer

In [ ]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy()

IOUmetric = sm.metrics.IOUScore()

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

metrics = ['accuracy', IOUmetric]

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Callbacks

In [ ]:
import os
from datetime import datetime

net = BACKBONE

callbacks = []

exps_dir = os.path.join(cwd, 'Training', 'Checkpoints', 'multiclass_segmentation_experiments' + net)
if not os.path.exists(exps_dir):
  os.makedirs(exps_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(exps_dir, 'weights'), 
                                                   save_weights_only=True, save_best_only=True)  # False to save the model directly

callbacks.append(ckpt_callback)

# Early Stopping
es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, mode='min')
callbacks.append(es_callback)

# Decay on Plateau
decay_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=decay, patience=5, verbose=1,
    mode='min', min_lr=min_lr)
callbacks.append(decay_callback)

# Training

In [ ]:
hist = model.fit(x=train_dataset,
                 epochs=100,
                 steps_per_epoch=len(dataset),
                 validation_data=valid_dataset,
                 validation_steps=len(dataset_valid),
                 callbacks=callbacks)

In [ ]:
plot_hist(hist)

# Prediction

In [ ]:
# First create the generator for the test_set

split_set(base_dir=test_dir, is_test = True)
temptest = os.path.join(test_dir, "temptest")

images = os.listdir(temptest)
submission_dict = {}

if len(images) > 0:
  idx = 0
  for img_name in images:

    imgs = Image.open(os.path.join(temptest, img_name))
    img_shp = imgs.size
    img = imgs.resize([img_w, img_h], resample=Image.NEAREST)
    img_arr = np.array(img)

    out_sigmoid = model.predict(x=tf.expand_dims(img_arr, 0))

    predicted_class = tf.argmax(out_sigmoid, -1)
    mask_arr = predicted_class[0, ...]
    mask_arr = np.array(mask_arr)

    fig, ax = plt.subplots(1, 3)
    ax[0].imshow(np.uint8(mask_arr))

    import cv2
    mask_arr = cv2.resize(mask_arr, dsize=img_shp, interpolation=cv2.INTER_NEAREST)
    ax[1].imshow(np.uint8(mask_arr))  
    ax[2].imshow(np.uint8(imgs))    

    plt.show()

    split_filename = img_name.split('_')
    curr_dataset = split_filename[0]
    curr_plant = split_filename[1]
    curr_plant = curr_plant.capitalize()

    img_name = os.path.splitext(img_name)
    img_name = img_name[0]

    submission_dict[img_name] = {}
    submission_dict[img_name]['shape'] = img_shp
    submission_dict[img_name]['team'] = curr_dataset
    submission_dict[img_name]['crop'] = curr_plant
    submission_dict[img_name]['segmentation'] = {}

    # RLE encoding
    # crop
    rle_encoded_crop = rle_encode(mask_arr == 1)
    # weed
    rle_encoded_weed = rle_encode(mask_arr == 2)

    submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
    submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

# Finally, save the results into the submission.json file
import json
with open('submission.json', 'w+') as f:
    json.dump(submission_dict, f)

In [ ]:
import zipfile
zipfile.ZipFile('submission.zip', mode='w').write("submission.json")